## Algorithm Steps (per “target”):

Each Postage Stamp Cube (PSC) is represented by the $m \times n$ matrix $P$ where $m$ represents the number of frames in the observation and $n$ the number of pixels in a frame. Note that we treat each stamp as a 1-D array and thus use a single index for traversing a stamp. There are a total of $p$ different PSC in an observation. The notation $P_{ij}^k$ would represent pixel $j$ in frame $i$ of stamp $k$.

The algorithm compares a target star to a number of reference stars. We will represent the target matrix as $T_0 = T$ and each reference as $P^k$. Each star in the image will take turns being the target and all other stars will be the reference for that run.

##### **Steps**

<a href="#" id="normalize"></a>

1. **Normalize**: Normalize each stamp $k$ to obtain $\hat{P^k}$, where each pixel $j$ in frame $i$ is given by: 
    
    $$ \hat{P_{ij}^k} = \frac{P_{ij}^k}{ \sum_{j=1}^{n}{P_{ij}^k} } $$
    
    for $(i = 0,...,m)$ frames. This normalized stamp has effectively marginalized the flux of the star and represents the shape of the star as it falls on the detectors. This shape will be compared both spatially and temporally in order to find "similar" stars.

<a href="#" id="find_reference"></a>

2. **Find Reference Stars**: Find other sources that have the same morphology as target by comparing normalized stamps. Comparison is a summed-squared difference between the pixels for each frame of the observation. That is, 

    $$ v_k = \sum_{i=1}^m{ \sum_{j=1}^n {(\hat{T}_{ij} - \hat{P_{ij}^k})^2} } $$

    where $(k=0,...,p)$ and $\hat{T}_{ij}$ and $\hat{P_{ij}^k}$ represent the pixel value at frame $i$ and index $j$ for the normalized target and reference $k$ respectively. We want to find the values of $k$ (which are index numbers) for which $v_k$ is a minimum, noting that when $\hat{T} = \hat{P^k}$ then $v_k = 0$ (i.e. a the target and the reference are perfect matches). 
    
    The ordered set of $v_k$ values is given as $V_l$ such that $V_0$ corresponds to the index number of the best match between $\hat{T}$ and $\hat{P}$ and a higher $l$ means there the index number is less of a match. Note that the index of $V$ is the same as the index numbers $k$ from $k=0,...,p$.
        
    This is performed for all references in $p$, which could potentially be minimized by some smart clustering of stars.

3. **Find Template Coeffecients**: A template is built using the normalized reference stars identified in the previous step, i.e. those with the lowest $v_k$ values. The template is created by determining the PSC-level coefficients ($c$) that minimize the least-square between the selected normalized references and the normalized target PSC. The selected references can be written as 

    $$ R = P_{V_0}, P_{V_1},...,P_{V_l} \; \mathrm{for} \; (l=0,...,r) $$
    
    for the regular stamps and 
    
    $$ \hat{R} = \hat{P}_{V_0}, \hat{P}_{V_1},...,\hat{P}_{V_l} \; \mathrm{for} \; (l=0,...,r) $$
    
    for the normalized versions. Currently the number of references $r$ used is manually selected but ranges from $r=50$ to $r=80$. A means of auto-determining the ideal $r$ remains to be done.
    
    Note that  $R_k \equiv P_k$ and $\hat{R}_k \equiv \hat{P}_k$ and that both $R$ and $\hat{R}$ are a $m \times n \times r$ matrix.  
    
    Our coefficients vector $c$ is also of length $r$ and we wish to obtain the solution to 
    
    $$ c \hat{R} = \hat{T} $$
    $$ \mathrm{or} $$
    $$ \mathtt{min} \| \hat{T} - c \hat{R} \|^2 $$

4. **Build Template**: The previous step used the normalized reference stamps across all $m$ observations to create one coeffecient per reference for each of the $l$ references in $\hat{R}$. To build the "ideal" PSC ($I$), we apply the same coeffecients to the non-normalized set of stamps:
    
    $$ I = cR. $$
        
    The ideal template $I$ is an $m \times n$ matrix where $I_i \approx T_i$ for each frame $i$ .

5. **Compare each frame**: We now compare each target frame $T_i$ to the ideal template frame $I_i$ to obtain a list of relative light curve values:

    $$ L = \frac{\sum_{j=0}^{n}{T_{ij}}}{\sum_{j=0}^{n}{I_{ij}}} \; \mathrm{for} \; (i=0,...,m). $$
    
    Note that the flux values between the target and the ideal PSC will likely be different and therefore the individual values of $I_i$ do not carry meaning. However, the _gradient_ of the line formed by the points in $L$ will be indicative of a relative change between $T_i$ and the "most similar" stars in the same frame.